In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Marie Curie received the Nobel Prize in Physics")
options = {"collapse_phrases": True}

# we can display
spacy.displacy.render(doc, style="dep", options=options, jupyter=True)
"""
(verb: receive, subject: p:Person, object: a:Prize) → (relationship: RECEIVE_PRIZE, from: p, to:a)

Here, “receive” (considered in the lemmatized version) is an English verb, whereas RECEIVE_PRIZE is a relationship type
(a semantic relation).
"""